In [2]:
import os
import django
import sys
from asgiref.sync import sync_to_async

sys.path.insert(0, '/home/mohammed/Desktop/tech_projects/growbal/growbal_django')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "growbal.settings")

django.setup()

from services.models import Service
from services.serializers import ServiceSerializer
from accounts.serializers import ServiceProviderProfileSerializer
from accounts.models import ServiceProviderProfile
from asgiref.sync import sync_to_async

@sync_to_async
def load_all_services():
    services = Service.objects.all()
    serializer = ServiceSerializer(services, many=True)
    return serializer.data


# @sync_to_async
# def load_all_service_provider_profiles():
#     profiles = ServiceProviderProfile.objects.all()
#     serializer = ServiceProviderProfileSerializer(profiles, many=True)
#     return serializer.data

@sync_to_async
def load_all_service_provider_profiles():
    profiles = ServiceProviderProfile.objects.prefetch_related("services").all()
    serialized_profiles = []

    for profile in profiles:
        # Serialize profile
        profile_data = ServiceProviderProfileSerializer(profile).data
        
        # Serialize linked services
        services = profile.services.all()
        services_data = ServiceSerializer(services, many=True).data
        
        # Include services in profile dictionary
        profile_data["services"] = services_data
        
        serialized_profiles.append(profile_data)
    
    return serialized_profiles

@sync_to_async
def update_service_general_descriptions(descriptions):
    services_to_update = Service.objects.filter(
        service_description=descriptions.company_service_description
    )

    services_to_update.update(
        general_service_description=descriptions.general_service_description
    )

X Establishment type should be set 0 for corporate or 1 individual
X Email is mandatory for user and establishment
X type_service_id must be set
X media -> media_collection -> user
media -> profile -> establishment
logo data population

In [5]:
# services = await load_all_services()
# for service in services:
#     print(service)

service_provider_profiles = await load_all_service_provider_profiles()
# print(service_provider_profiles[0].keys())
# for profile in service_provider_profiles:
#     print(profile["services"])

In [6]:
from collections import defaultdict

name_counts = defaultdict(list)
for profile in service_provider_profiles:
    name_counts[profile["name"]].append(profile)

duplicates = {name: profiles for name, profiles in name_counts.items() if len(profiles) > 1}

# Output the duplicates
for name, profiles in duplicates.items():
    print(f"Duplicate Name: {name}")
    for profile in profiles:
        print(profile)
    print()


In [12]:
import pandas as pd
import mysql.connector as mc

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── QUERY DATA FROM 'establishment' TABLE ORDERED BY creation_date ──
cur.execute(f"""
    SELECT * FROM `{SCHEMA}`.`establishment`
    ORDER BY creation_date DESC;
""")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_establishment = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ────────────────────────────────────────────
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)

print(f"Total rows: {len(df_establishment)}")

# print(df_establishment["slug"].unique().tolist())

df_establishment = df_establishment.head(60)
display(df_establishment.head(5))

# ── CLOSE CONNECTION ────────────────────────────────────────────────
cur.close()
cnx.close()


Total rows: 262


,id,adress,blocked,company_name,creation_date,deleted,description,designation,email,expiry_date,founded_in,password,phone,representative_name,slug,sub_name,tax_number,type,verified,web_site,city_id,country_id,cover_picture_id,licence_id,logo_id,profile_id,region_id,type_ez_id,user_user_id,agency,vision,mobile
0,433,"Office No. 2501 - 2506, 25th Floor, Burj Al Salam, Trade Center First, Sh. Zayed Road, Dubai, UAE",0,BFG Advisory,2025-06-13 07:56:36.119,0,BFG Advisory provides comprehensive Golden Visa services in the UAE with a 100% success rate. The service includes e...,None,info@bfgadvisory.com,None,None,None,+971502130288,None,DMuejeEe,None,None,0.0,0,https://bfgadvisory.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,+971 50 213 0327
1,432,"Office 105, Level 1, Emaar Square – Building 4, Sheikh Mohammed Bin Rashid Boulevard Downtown Dubai, United Arab Emi...",0,Creative Zone,2025-06-13 07:56:36.117,0,"Creative Zone is a leading business setup advisor in the Middle East, providing comprehensive company formation serv...",None,setup@creativezone.ae,None,None,None,800-542-3673,None,FY64Nwwy,None,None,0.0,0,https://www.creativezone.ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,+971 800 5423673
2,431,Amer Center Al Tawar – Building – 1E – 155 Al Quds St – Al Twar – Dubai Airport Free Zone – Dubai – United Arab Emir...,0,Amer Center Al Twar,2025-06-13 07:56:36.116,0,"Amer Center provides comprehensive government services in Dubai, specializing in visa processing, residency applicat...",None,main@businessindxb.com,None,None,None,None,None,lkRqff6w,None,None,0.0,0,https://amer.center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,+971504512311
3,430,"Cluster W, JLT - Dubai",0,Aston VIP,2025-06-13 07:56:36.114,0,"24-Hour Dubai residence VISA service with expedited processing. The service includes initial consultation, document ...",None,contact@aston.ae,None,None,None,+971 4 518 2623,None,TZ8BDKNW,None,None,0.0,0,https://aston.ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,429,Dubai Land Department Headquarters,0,Dubai Land Department,2025-06-13 07:56:36.111,0,Dubai Land Department (DLD) provides real estate valuation services through accredited valuation companies. These co...,None,info@dubailand.gov.ae,None,None,None,97142438777,None,gI2r4qoK,None,None,0.0,0,https://dubailand.gov.ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,0585120063


In [2]:
import pandas as pd
import mysql.connector as mc

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`service`;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
print(len(df_service))
# df_service = df_service.head(100)
display(df_service.tail(5))

156


,id,description,name,type_establishment,image_id,type_service_id,offre_id,deleted
151,154,None,UAE Government Services,NaN,NaN,1,None,0
152,155,None,Emirates ID Typing Service,NaN,NaN,1,None,0
153,156,None,Business Setup and Company Formation Services in Dubai,NaN,NaN,1,None,0
154,157,None,Dubai Business Setup and Company Formation Services,NaN,NaN,1,None,0
155,158,None,UAE Golden Visa Services,NaN,NaN,1,None,0


In [3]:
import pandas as pd
import mysql.connector as mc
import string
import random

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)
# dict_keys(['user', 'provider_type', 'country', 'session_status', 'name', 'logo', 'website', 'linkedin', 'facebook', 'instagram', 'telephone', 'mobile', 'emails', 'office_locations', 'key_individuals', 'services'])

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`media`;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
# print(df_service["path"][0])
print(len(df_service))
# print(df_service["content_type"].unique())
# df_service = df_service.head(100)
display(df_service.head(5))

2679


,id,content_type,creation_date,deleted,label,path,document_collection_media_collection_id,media_collection_id,user_media_user_id,media_user_creation_user_id,message_id,message_ordre_id,profile_id,country_over_view_id,task_id
0,1,image/png,2023-12-25 15:04:54.135,0,avatar agent 220231225160454135.png,https://staging.growbal.net:8081/gateway/cms/media/display_image/1,NaN,106.0,None,None,NaN,None,None,None,NaN
1,2,image/png,2023-12-25 20:25:44.397,0,avatar EZ 120231225212544395.png,https://staging.growbal.net:8081/gateway/cms/media/display_image/2,NaN,NaN,None,None,NaN,None,None,None,NaN
2,3,image/png,2023-12-25 20:37:59.363,0,avatar EZ 120231225213759361.png,https://staging.growbal.net:8081/gateway/cms/media/display_image/3,NaN,NaN,None,None,NaN,None,None,None,NaN
3,4,image/png,2023-12-25 20:39:33.215,0,avatar EZ 120231225213933207.png,https://staging.growbal.net:8081/gateway/cms/media/display_image/4,NaN,NaN,None,None,NaN,None,None,None,NaN
4,5,image/png,2023-12-25 20:48:12.370,0,avatar EZ 220231225214812366.png,https://staging.growbal.net:8081/gateway/cms/media/display_image/5,NaN,NaN,None,None,NaN,None,None,None,NaN


In [5]:
import pandas as pd
import mysql.connector as mc
import string
import random

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)
# dict_keys(['user', 'provider_type', 'country', 'session_status', 'name', 'logo', 'website', 'linkedin', 'facebook', 'instagram', 'telephone', 'mobile', 'emails', 'office_locations', 'key_individuals', 'services'])

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`media_collection` ORDER BY creation_date DESC;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
# print(df_service["path"][0])
print(len(df_service))
# print(df_service["content_type"].unique())
# df_service = df_service.head(100)
display(df_service.head(5))

304


,media_collection_id,creation_date,deleted,description,label,type,user_id,profile_id
0,265,2025-06-13 07:56:36,0,None,None,None,373.0,NaN
1,266,2025-06-13 07:56:36,0,None,None,None,374.0,NaN
2,267,2025-06-13 07:56:36,0,None,None,None,375.0,NaN
3,268,2025-06-13 07:56:36,0,None,None,None,376.0,NaN
4,269,2025-06-13 07:56:36,0,None,None,None,377.0,NaN


In [12]:
import os
import shutil
from datetime import datetime
import mysql.connector as mc
import mimetypes

HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

def insert_media_collection(user_id, cursor):
    cursor.execute(f"""
        SELECT media_collection_id FROM `{SCHEMA}`.`media_collection`
        WHERE user_id = %s
    """, (user_id,))

    existing_collection = cursor.fetchone()

    if existing_collection:
        return False

    creation_date = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    insert_query = f"""
        INSERT INTO `{SCHEMA}`.`media_collection` (
            user_id, creation_date, deleted
        ) VALUES (%s, %s, %s)
    """

    values = (
        user_id,
        creation_date,
        0
    )

    cursor.execute(insert_query, values)
    collection_id = cursor.lastrowid

    return collection_id

def insert_media_record(absolute_logo_path, logos_dest_dir, media_collection_id, content_type, cursor):
    file_name = os.path.basename(absolute_logo_path)

    # Check for existing record
    cursor.execute(f"""
        SELECT id FROM `{SCHEMA}`.`media`
        WHERE label = %s
    """, (file_name,))

    existing_media = cursor.fetchone()

    if existing_media:
        return False

    creation_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

    insert_query = f"""
        INSERT INTO `{SCHEMA}`.`media` (
            content_type, creation_date, deleted, label, media_collection_id
        ) VALUES (%s, %s, %s, %s, %s)
    """

    values = (
        content_type,
        creation_date,
        0,
        file_name,
        media_collection_id
    )

    cursor.execute(insert_query, values)
    media_id = cursor.lastrowid

    # Prepare new file path using media ID and original extension
    file_extension = os.path.splitext(file_name)[1]
    new_file_name = f"{media_id}{file_extension}"
    new_file_path = os.path.join(logos_dest_dir, new_file_name)

    # Update path in database
    cursor.execute(f"""
        UPDATE `{SCHEMA}`.`media`
        SET path = %s
        WHERE id = %s
    """, (new_file_path, media_id))

    # Copy file to the new destination
    # os.makedirs(logos_dest_dir, exist_ok=True)
    if not os.path.exists(logos_dest_dir):
        print(f"Creating destination directory: {logos_dest_dir}")
        os.makedirs(logos_dest_dir)
        print(f"destination directory: {logos_dest_dir} created")
    shutil.copy2(absolute_logo_path, new_file_path)

    return media_id

absolute_logo_path = "/home/mohammed/Desktop/tech_projects/growbal/growbal_django/media/logos/logo-placeholder.png"
logos_dest_dir = "/home/mohammed/Desktop/tech_projects/growbal/growbal_django/media/logos_test"
user_id = 1

cnx = mc.connect(
    host=HOST, port=PORT, user=USER, password=PASSWORD,
    database=SCHEMA, auth_plugin="mysql_native_password"
)
cur = cnx.cursor()

content_type, _ = mimetypes.guess_type(absolute_logo_path)
collection_id = insert_media_collection(user_id, cur)
if collection_id:
    media_id = insert_media_record(absolute_logo_path, logos_dest_dir, collection_id, content_type, cur)
    if media_id:
        print(f"Successfully inserted media record: {media_id}")
        cnx.commit()
    else:
        print("Failed to insert media record")
        cnx.rollback()

Successfully inserted media record: 2684


In [13]:
print(collection_id)

267


In [53]:
import os
import mimetypes


def print_image_content_types(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            content_type, _ = mimetypes.guess_type(filepath)
            print(f"{filename}: {content_type}")


# Usage example
download_dir = "/home/mohammed/Desktop/tech_projects/growbal/growbal_django/media/logos"
print_image_content_types(download_dir)

mof-logo.svg_0.svg: image/svg+xml
logo-placeholder.png: image/png
arb-logo.png_0.png: image/png
vz-ascentium-1.svg_0.svg: image/svg+xml
vz-ascentium-1.svg_1.svg: image/svg+xml
BSE.png_0.png: image/png
CIG-LOGO-01-1-1.png_0.png: image/png
logo-fta.webp_0.img: None


In [2]:
import pandas as pd
import mysql.connector as mc
import string
import random

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)
# dict_keys(['user', 'provider_type', 'country', 'session_status', 'name', 'logo', 'website', 'linkedin', 'facebook', 'instagram', 'telephone', 'mobile', 'emails', 'office_locations', 'key_individuals', 'services'])

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`user`;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
print(len(df_service))
# df_service = df_service.head(100)
display(df_service.head(5))

226


,user_id,adress,birthday,creation_date,email,full_name,is_deleted,nationality,password,phone,role,stars,web_site,establishment_id,image_id,passeport_copy_id,provider,provider_id,confirm_email
0,1,None,1995-12-29 14:58:56.073000,2023-12-25 15:05:09.628,growbal.admin@gmail.com,GROWBAL ADMIN,0,b'\xac\xed\x00\x05sr\x00\x13java.util.ArrayListx\x81\xd2\x1d\x99\xc7a\x9d\x03\x00\x01I\x00\x04sizexp\x00\x00\x00\x01...,$2a$10$N2yN5vanlpviP58OXN3KFeMZqWVI.EsmBrOs9YwCA3XC9CBXNJomC,+21631572542,1,0.0,None,1,1.0,NaN,None,None,1
1,2,None,None,2023-12-25 21:17:40.801,admin.jafza@growbal.com,None,0,None,$2a$10$cPmfRPFGz/m/QV1c/iVbKOsi6hvEPOvN/8ZWXRZdwCkV6AYBo9a.C,+216 4 4453270,6,0.0,None,2,NaN,NaN,None,None,1
2,5,None,0985-12-20 00:00:00,2023-12-26 13:35:10.327,valentina.otero@ohllp.com,Valentina Otero,0,b'\xac\xed\x00\x05sr\x00\x13java.util.ArrayListx\x81\xd2\x1d\x99\xc7a\x9d\x03\x00\x01I\x00\x04sizexp\x00\x00\x00\x01...,$2a$10$5RIta34cyNct9vXB.JNDWeSbcCozRX3qcKLVCgz/F3339N41jhAhu,+21650 189 7506,4,5.0,None,5,23.0,24.0,None,None,1
3,6,None,None,2023-12-26 13:53:05.037,customersupport@shams.ae,None,0,None,$2a$10$5WMlwIzIDnoLK0wlJ2bmd.Ke6S99qqWpSTzMVv5XGOZxzjFErN5cS,+216 80074267,6,0.0,None,6,NaN,NaN,None,None,1
4,7,None,None,2023-12-26 14:06:15.235,blz@growbal.com,None,0,None,$2a$10$vdxA8Oxo7fdm1TkN5JNo1uQ7BQIZSDN4Wkqwi.YlaYorsAnSLRyA2,+973 17589999,6,0.0,None,7,NaN,NaN,None,None,1


In [4]:
import pandas as pd
import mysql.connector as mc
import string
import random

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)
# dict_keys(['user', 'provider_type', 'country', 'session_status', 'name', 'logo', 'website', 'linkedin', 'facebook', 'instagram', 'telephone', 'mobile', 'emails', 'office_locations', 'key_individuals', 'services'])

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`type_ez`;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
# print(df_service["path"][119])
print(len(df_service))
# df_service = df_service.head(100)
display(df_service.tail(5))

3


,id,deleted,type
0,1,0,Economic Zone
1,2,0,Free Zone
2,3,0,Logistic Zone


In [ ]:
import pandas as pd
import mysql.connector as mc

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── QUERY DATA FROM 'service' TABLE ──────────────────────────
cur.execute(f"SELECT * FROM `{SCHEMA}`.`establishment_services`;")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_service = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ──────────────────────────────────────────────
pd.set_option("display.max_columns", None)
# ── DISPLAY OPTIONS ────────────────────────────────────────────────
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)
print(len(df_service))
# df_service = df_service.head(100)
display(df_service.tail(50))

In [6]:
import pandas as pd
import mysql.connector as mc

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── QUERY DATA FROM 'user' TABLE ORDERED BY creation_date ──
cur.execute(f"""
    SELECT * FROM `{SCHEMA}`.`user`
    ORDER BY creation_date DESC;
""")
data = cur.fetchall()

# ── DISPLAY DATA AS DATAFRAME ──────────────────────────────────────
df_user = pd.DataFrame(data)

# ── DISPLAY ALL COLUMNS ────────────────────────────────────────────
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 120)

print(f"Total rows: {len(df_user)}")
df_user = df_user.head(10)
display(df_user.tail(5))
# display(df_user)

# ── CLOSE CONNECTION ────────────────────────────────────────────────
cur.close()
cnx.close()

Total rows: 226


,user_id,adress,birthday,creation_date,email,full_name,is_deleted,nationality,password,phone,role,stars,web_site,establishment_id,image_id,passeport_copy_id,provider,provider_id,confirm_email
5,375,"Ministry of Finance, United Arab Emirates",None,2025-06-04 07:18:37.449,info@mof.gov.ae,Ministry of Finance - United Arab Emirates,0,None,None,047771000,4,0.0,https://mof.gov.ae,392,NaN,NaN,None,None,1
6,374,UAE Embassy in London,None,2025-06-04 07:18:37.447,None,UAE Embassy in London,0,None,None,+97180044444,4,0.0,https://www.mofa.gov.ae/en/Missions/London,391,NaN,NaN,None,None,1
7,373,United Arab Emirates,None,2025-06-04 07:18:37.445,None,Ministry of Economy UAE,0,None,None,800 1222,4,0.0,https://www.moec.gov.ae,390,NaN,NaN,None,None,1
8,372,None,None,2025-01-27 13:47:31.172,42dcf56819@emailawb.pro,None,0,None,$2a$10$UmX/hIeTkMUyNt/172kY5OqLazDhKdbbpOfL8/ILaaQr8/9cSPMC.,+21620025557,6,3.0,None,389,NaN,NaN,None,None,1
9,371,None,None,2025-01-27 13:44:07.627,789abe94bf@emailawb.pro,None,0,None,$2a$10$notBqxDfHp.cT31nmtfMue06rcgtLw7/N1OmyYL.NNtqNnnvk0/ki,+21620036366,6,3.0,None,388,NaN,NaN,None,None,1


In [4]:
from fuzzywuzzy import fuzz, process

# Example structure of your existing data:
# df_service contains column "company_name"
service_names = df_service["name"].tolist()

# Function to filter similar matches
def get_potential_matches(name, service_names, threshold=70):
    matches = process.extract(name, service_names, scorer=fuzz.token_set_ratio)
    return [match for match, score in matches if score >= threshold]

# Iterate over profiles to build potential matches
profile_potential_matches = {}
for service in services:
    service_name = service["service_title"]
    similar_names = get_potential_matches(service_name, service_names, threshold=70)
    profile_potential_matches[service_name] = similar_names

for service, similar_names in profile_potential_matches.items():
    print(f"{service}: {similar_names}")

Management Consulting & Professional Services: ['Consulting', 'Consulting', 'Consulting Services', 'Consulting ', 'Customs Consulting & Services']
Tax Preparation and Financial Services: ['Port Services', 'Legal Services']
Tax Preparation and Accounting Services: ['Port Services', 'Legal Services']
Professional Tax Services: ['Port Services', 'Legal Services']
Global Management Consulting Services: ['Consulting', 'Consulting', 'Consulting Services', 'Consulting ', 'Customs Consulting & Services']
Management Consulting & Digital Transformation Services: ['Consulting', 'Consulting', 'Consulting Services', 'Consulting ', 'Customs Consulting & Services']
Professional Tax and Accounting Services: ['Port Services', 'Legal Services']
Strategy and Management Consulting for Financial Services: ['Consulting', 'Consulting', 'Consulting Services', 'Consulting ', 'Customs Consulting & Services']
Real Estate Tax and Accounting Services: ['Real Estate', 'Real Estate', 'Real Estate Services', 'Real es

In [25]:
from fuzzywuzzy import fuzz, process

service_descriptions = df_service["description"].tolist()

# Function to filter similar matches
def get_potential_matches(name, service_descriptions, threshold=70):
    matches = process.extract(name, service_descriptions, scorer=fuzz.token_set_ratio)
    return [match for match, score in matches if score >= threshold]

# Iterate over profiles to build potential matches
profile_potential_matches = {}
for service in services:
    service_description = service["service_description"]
    similar_descriptions = get_potential_matches(service_description, service_descriptions, threshold=60)
    profile_potential_matches[service_description] = similar_descriptions

# for service_description, similar_descriptions in profile_potential_matches.items():
#     if similar_descriptions:
#         print(f"{service_description}: {similar_descriptions}")

for service_description in service_descriptions:
    if service_description:
        if "tax" in service_description or "Tax" in service_description or "TAX" in service_description:
            print(service_description)
        if "IT" in service_description:
            print(service_description)
        if "legal" in service_description or "Legal" in service_description or "LEGAL" in service_description:
            print(service_description)
        
    

Legal
Legal
Legal services encompass a diverse array of offerings provided by qualified legal professionals, including attorneys, lawyers, paralegals, and legal consultants.
Legal services encompass a wide range of offerings designed to address the legal needs and concerns of individuals, businesses, and organizations. 
Legal services encompass a wide range of offerings designed to address the legal needs and concerns of individuals, businesses, and organizations. 
Legal services encompass a wide range of offerings designed to address the legal needs and concerns of individuals, businesses, and organizations. 
A range of legal tasks, such as drafting legal documents and providing legal advice on various matters.
A range of legal tasks, such as drafting legal documents and providing legal advice on various matters. 
Handle administrative tasks related to payroll, taxes, and immigration.
Enable citizens or permanent residents of a country to sponsor eligible family members for immigratio

In [26]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
description_list = []
for text in service_descriptions:
    if text:
        description_list.append(text)

token_counts = [len(encoding.encode(text)) for text in description_list]
average_tokens = sum(token_counts) / len(token_counts)

print(f"Average number of tokens: {average_tokens:.2f}")

Average number of tokens: 13.43


In [5]:
from fuzzywuzzy import fuzz, process

# Example structure of your existing data:
# df_establishment contains column "company_name"
establishment_names = df_establishment["company_name"].tolist()

# Function to filter similar matches
def get_potential_matches(name, establishment_names, threshold=70):
    matches = process.extract(name, establishment_names, scorer=fuzz.token_set_ratio)
    return [match for match, score in matches if score >= threshold]

# Iterate over profiles to build potential matches
profile_potential_matches = {}
for profile in service_provider_profiles:
    company_name = profile["name"]
    similar_names = get_potential_matches(company_name, establishment_names, threshold=70)
    profile_potential_matches[company_name] = similar_names

for profile, similar_names in profile_potential_matches.items():
    print(f"{profile}: {similar_names}")


Alvarez & Marsal: []
Ravi Tax Preparation Services Inc: []
Tax Comp: []
TAXtical LLC: []
NMS Consulting: []
Cambridge Management Consulting: []
FBC & TAX SERVICES: []
zeb: []
GavTax Advisory Services: []
RSM US LLP: []
BDO Global: []
KLR: []
Tata Consultancy Services Limited: []
Gulf Coast Accounting & Tax Services: []
MNP LLP: []
McKinsey & Company: []
Aon: []
Singapore Chartered Tax Professionals Limited: []
Oregon Association of Tax Consultants: []
Paradigm Business Solutions LLC: []
Cash Tracks Financial: []
Duke A. Forbes CPA, A Professional Corporation: []
EthioDiasporaHub: []
Aegon: []
Blick Rothenberg: []
Grant Thornton Ireland: []
Atos: []
Wolters Kluwer: []
Intalio: []
Rimini Street: []
The Leppert CPA Group: []
Qualys: []
WWT (World Wide Technology): []
Nuvei: []
Abdiwali Mohamed CPA PLLC: []
Barrett Distribution Centers: []
Tinuiti: []
Aprio: []
Jitendra Chartered Accountants: []
Mahanakorn Partners Group: []
Lazard: []
Column Tax: []
U.S. Department of Commerce: []
Tax990:

In [ ]:
import mysql.connector as mc
from datetime import datetime
from mysql.connector.errors import IntegrityError

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)
# dict_keys(['user', 'provider_type', 'country', 'session_status', 'name', 'logo', 'website', 'linkedin', 'facebook', 'instagram', 'telephone', 'mobile', 'emails', 'office_locations', 'key_individuals', 'services'])

def truncate_string(value, max_length):
    if value is None:
        return None
    return value[:max_length]
# ── INSERT FUNCTION ────────────────────────────────────────────────
def insert_establishment(profile_dict, cursor):
    insert_query = f"""
    INSERT INTO `{SCHEMA}`.`establishment` (
        mobile, email, web_site, phone, description, creation_date, company_name, adress, blocked, deleted, verified
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Prepare values
    values = (
        profile_dict.get('mobile') if profile_dict.get('mobile') else None,
        profile_dict.get('emails')[0] if profile_dict.get('emails') else None,
        profile_dict.get('website') if profile_dict.get('website') else None,
        profile_dict.get('telephone') if profile_dict.get('telephone') else None,
        profile_dict["services"][0].get('service_description') if profile_dict.get('services') else None,
        datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3],
        profile_dict.get('name') if profile_dict.get('name') else None,
        # profile_dict.get('office_locations') if profile_dict.get('office_locations') else None,
        truncate_string(profile_dict.get('office_locations'), 255) if profile_dict.get('office_locations') else None,
        False,
        False,
        False
    )
    for value in values:
        print(type(value))
    print(values)

    # Execute query
    cursor.execute(insert_query, values)
    establishment_id = cursor.lastrowid
    return establishment_id

# ── INSERT FUNCTION FOR SERVICE TABLE ──────────────────────────────
def insert_service(service_dict, cursor):
    insert_query = f"""
    INSERT INTO `{SCHEMA}`.`service` (
        name, description, deleted
    )
    VALUES (%s, %s, %s)
    """

    # Prepare values
    values = (
        service_dict.get('service_title'),
        service_dict.get('general_service_description'),
        False
    )

    print(f"Inserting: {values}")

    # Execute query
    cursor.execute(insert_query, values)
    service_id = cursor.lastrowid
    return service_id

# ── INSERT FUNCTION FOR ESTABLISHMENT_SERVICE TABLE ──────────────────────────────
def insert_establishment_service(establishment_id, service_id, cursor):
    insert_query = f"""
    INSERT INTO `{SCHEMA}`.`establishment_services` (
        establishment_id, service_id, deleted
    )
    VALUES (%s, %s, %s)
    """

    # Prepare values
    values = (
        establishment_id,
        service_id,
        False
    )

    print(f"Inserting establishment_service record: {values}")

    # Execute query
    cursor.execute(insert_query, values)
    record_id = cursor.lastrowid
    return record_id

# ── INSERT FUNCTION FOR user TABLE ──────────────────────────────
def insert_user(profile_dict, establishment_id, cursor):
    insert_query = f"""
    INSERT INTO `{SCHEMA}`.`user` (
        adress, creation_date, email, full_name, phone, web_site, establishment_id, is_deleted, stars, confirm_email
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Prepare values
    values = (
        truncate_string(profile_dict.get('office_locations'), 255) if profile_dict.get('office_locations') else None,
        datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3],
        profile_dict.get('emails')[0] if profile_dict.get('emails') else None,
        profile_dict.get('name') if profile_dict.get('name') else None,
        profile_dict.get('telephone') if profile_dict.get('telephone') else None,
        profile_dict.get('website') if profile_dict.get('website') else None,
        establishment_id,
        False,
        0,
        1
    )

    print(f"Inserting user record: {values}")

    # Execute query
    cursor.execute(insert_query, values)
    record_id = cursor.lastrowid
    return record_id

# # ── INSERT ALL PROFILES ────────────────────────────────────────────
# for profile in service_provider_profiles:
#     insert_establishment(profile, cur)
#     # break

# try:
#     inserted_count = 0
#     skipped_count = 0
#     for profile in service_provider_profiles:
#         # if profile.get('name') == "Atos":
#         #     print("########################################")
#         #     print(profile)
#         #     print("########################################")
#         try:
#             insert_establishment(profile, cur)
#             inserted_count += 1
#             print(f"Inserted {inserted_count} profiles...")
#         except IntegrityError as e:
#             if e.errno == 1062:
#                 skipped_count += 1
#                 print(f"Skipped duplicate entry: {profile.get('name')}")
#             else:
#                 print(f"Encountered unexpected IntegrityError: {e}")
#                 raise e  # re-raise unexpected errors
#     print("All insertions succeeded without errors. Rolling back test inserts.")
# except Exception as e:
#     # print(f"Error encountered: {e}")
#     raise e
# finally:
#     # Single rollback to clear all test inserts at once
#     cnx.rollback()
#     cur.close()
#     cnx.close()

try:
    inserted_count = 0
    skipped_count = 0
    for profile in service_provider_profiles:
        try:
            establishment_id = insert_establishment(profile, cur)
            for service in profile["services"]:
                service_id = insert_service(service, cur)
                insert_establishment_service(establishment_id, service_id, cur)
            inserted_count += 1
            insert_user(profile, establishment_id, cur)
            print(f"Inserted {inserted_count} profiles...")
        except IntegrityError as e:
            if e.errno == 1062:
                skipped_count += 1
                print(f"Skipped duplicate entry: {profile.get('name')}")
            else:
                print(f"Encountered unexpected IntegrityError: {e}")
                raise e
    
    cnx.commit()  # <-- Commit your changes explicitly
    print(f"Final insertion complete. Total inserted: {inserted_count}, duplicates skipped: {skipped_count}")

except Exception as e:
    cnx.rollback()  # Rollback on unexpected exceptions
    print(f"Error encountered, rolled back changes: {e}")
    raise e

finally:
    cur.close()
    cnx.close()

print(f"Inserted {inserted_count} profiles.")
print(f"Skipped {skipped_count} duplicate entries.")

Inserting user record: ('Thales Headquarters location', '2025-06-02 07:31:45.397', None, 'Thales Group', '+33 (0) 1 57 77 80 00', 'https://www.thalesgroup.com', 1, False, 0, 1)
Inserted 1 profiles...
Inserting user record: ('Floor 16, Bay Gate Tower, Business Bay P.O.Box 88007, Dubai – United Arab Emirates', '2025-06-02 07:31:45.398', 'info@dubaiconsultancy.ae', 'Dubai Consultancy LLC', '+971 4 554 2020', 'https://dubaiconsultancy.ae/', 1, False, 0, 1)
Inserted 2 profiles...
Inserting user record: ('27451 Tourney Road, Ste. 160, Valencia, California 91355', '2025-06-02 07:31:45.399', 'info@protaxservices.com', 'Professional Tax Services', '(661) 259-1967', 'https://protaxservices.com', 1, False, 0, 1)
Inserted 3 profiles...
Inserting user record: ('84 offices across 40 countries spanning 6 continents', '2025-06-02 07:31:45.399', None, 'Alvarez & Marsal', None, 'https://www.alvarezandmarsal.com', 1, False, 0, 1)
Inserted 4 profiles...
Inserting user record: ('Global Headquarters: 280 Pa

In [3]:
import mysql.connector as mc
from datetime import datetime
from mysql.connector.errors import IntegrityError

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST     = "127.0.0.1"
PORT     = 3306
USER     = "root"
PASSWORD = "rootpass"
SCHEMA   = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── INSERT FUNCTION FOR SERVICE TABLE ──────────────────────────────
def insert_service(service_dict, cursor):
    insert_query = f"""
    INSERT INTO `{SCHEMA}`.`service` (
        name, description, deleted
    )
    VALUES (%s, %s, %s)
    """

    # Prepare values
    print("###############################")
    print(service_dict.get('general_service_description'))
    values = (
        service_dict.get('service_title'),
        service_dict.get('general_service_description'),
        False
    )

    print(f"Inserting: {values}")

    # Execute query
    cursor.execute(insert_query, values)

try:
    inserted_count = 0
    skipped_count = 0

    for service in services:
        print(service.get('general_service_description'))
    #     try:
    #         insert_service(service, cur)
    #         inserted_count += 1
    #         print(f"Inserted {inserted_count} services...")
    #     except IntegrityError as e:
    #         if e.errno == 1062:
    #             skipped_count += 1
    #             print(f"Skipped duplicate entry: {service.get('service_title')}")
    #         else:
    #             print(f"Encountered unexpected IntegrityError: {e}")
    #             raise e

    # cnx.commit()  # <-- Commit your changes explicitly
    # print(f"Final insertion complete. Total inserted: {inserted_count}, duplicates skipped: {skipped_count}")

except Exception as e:
    cnx.rollback()  # Rollback on unexpected exceptions
    print(f"Error encountered, rolled back changes: {e}")
    raise e

finally:
    cur.close()
    cnx.close()

print(f"Inserted {inserted_count} services.")
print(f"Skipped {skipped_count} duplicate entries.")

Executive-level advisory services encompassing strategic planning, communications, financial guidance, management consulting, risk management, and human capital solutions. Services focus on facilitating strategic organizational change, providing cross-functional expertise during critical business transitions, and delivering comprehensive business transformation support.
Tax preparation and consulting services including tax return preparation, tax planning, and professional tax advisory support for individuals and businesses.
Comprehensive tax and financial services encompassing tax preparation, accounting, bookkeeping, and tax problem resolution. Services include professional tax preparation for corporate, estate, individual, partnership, and multi-jurisdictional taxes. The service provides personalized tax preparation and planning, maintaining compliance with current tax laws and adapting to various life events.
Strategic M&A advisory services focusing on deal strategy execution and v

In [16]:
import pandas as pd
import mysql.connector as mc

# ── DB CONFIG ──────────────────────────────────────────────────────
HOST = "127.0.0.1"
PORT = 3306
USER = "root"
PASSWORD = "rootpass"
SCHEMA = "growbal"

# ── CONNECT ────────────────────────────────────────────────────────
cnx = mc.connect(
    host=HOST, port=PORT,
    user=USER, password=PASSWORD,
    database=SCHEMA,
    auth_plugin="mysql_native_password",
)
cur = cnx.cursor(dictionary=True)

# ── EXECUTE QUERY ──────────────────────────────────────────────────
cur.execute(f"DESCRIBE `{SCHEMA}`.`establishment`;")
schema_info = cur.fetchall()

# ── DISPLAY RESULTS AS DATAFRAME ───────────────────────────────────
df_schema = pd.DataFrame(schema_info)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

print(df_schema)

# ── CLOSE CONNECTION ───────────────────────────────────────────────
cur.close()
cnx.close()


                  Field          Type Null  Key Default           Extra
0                    id        bigint   NO  PRI    None  auto_increment
1                adress  varchar(255)  YES         None                
2               blocked        bit(1)   NO         None                
3          company_name  varchar(255)  YES  UNI    None                
4         creation_date   datetime(6)  YES         None                
5               deleted        bit(1)   NO         None                
6           description      longtext  YES         None                
7           designation  varchar(255)  YES         None                
8                 email  varchar(255)  YES  UNI    None                
9           expiry_date   datetime(6)  YES         None                
10           founded_in   datetime(6)  YES         None                
11             password  varchar(255)  YES         None                
12                phone  varchar(255)  YES         None         

In [4]:
services = await load_all_services()
company_service_descriptions = []
for service in services:
    if service["service_description"]:
        company_service_descriptions.append(service["service_description"])
print(company_service_descriptions)

['A&M is a global professional services firm that provides advisory, business performance improvement and turnaround management services across six continents. They help organizations tackle complex business issues, boost operating performance and maximize stakeholder value. The firm specializes in providing practical solutions to unique problems of companies, investors and government entities.', 'Comprehensive tax preparation and financial services including business services (payroll, bookkeeping, sales & use tax), individual tax services, incorporation services (Corporation, LLC, Non-Profit), financial planning (retirement, college & education, home owners), real estate & loans, and insurance services (personal, business, health, life).', 'Tax Comp has been serving the communities of Moreno Valley and Riverside area for 40 years, providing professional tax preparation, planning, and IRS tax problem resolution services. The firm specializes in both personal and business tax return pr

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
description_list = []
for text in company_service_descriptions:
    if text:
        description_list.append(text)

token_counts = [len(encoding.encode(text)) for text in description_list]
print(sum(token_counts))
average_tokens = sum(token_counts) / len(token_counts)

print(f"Average number of tokens: {average_tokens:.2f}")

5116
Average number of tokens: 73.09


In [5]:
from pydantic import BaseModel
from langchain_anthropic import ChatAnthropic
from anthropic._exceptions import OverloadedError
from langchain_core.prompts import ChatPromptTemplate
from typing import List

class GeneralServiceDescription(BaseModel):
    company_service_description: str
    general_service_description: str

class GeneralServiceDescriptionOutput(BaseModel):
    general_service_description_output: List[GeneralServiceDescription]

def load_prompt(prompt_name):
    with open(f"prompts/{prompt_name}.md", "r") as f:
        return f.read()

prompt = load_prompt("generate_general_service_descriptions")

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("human", "Generate general service descriptions from the given company specific service descriptions")
])

# llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model=llm_model, temperature=0).with_structured_output(RelevanceCheckOutput)
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    anthropic_api_key=os.environ["ANTHROPIC_API_KEY"], temperature=0).with_structured_output(GeneralServiceDescriptionOutput)
llm_chain = prompt_template | llm
llm_chain = llm_chain.with_retry(
    retry_if_exception_type=(OverloadedError,),
    wait_exponential_jitter=True,    
    stop_after_attempt=12
)

In [9]:
general_service_descriptions = None
batch_size = 5
for i in range(0, len(company_service_descriptions), batch_size):
    print(f"Processing batch {i//batch_size+1} of {len(company_service_descriptions)//batch_size}")
    print(f"{i} to {i+batch_size}")
    general_service_descriptions_response = llm_chain.invoke({"company_service_descriptions": company_service_descriptions[i:i+batch_size]})
    if general_service_descriptions is None:
        general_service_descriptions = general_service_descriptions_response
    else:
        general_service_descriptions.general_service_description_output.extend(general_service_descriptions_response.general_service_description_output)

Processing batch 1 of 54
0 to 5
Processing batch 2 of 54
5 to 10
Processing batch 3 of 54
10 to 15
Processing batch 4 of 54
15 to 20
Processing batch 5 of 54
20 to 25
Processing batch 6 of 54
25 to 30
Processing batch 7 of 54
30 to 35
Processing batch 8 of 54
35 to 40
Processing batch 9 of 54
40 to 45
Processing batch 10 of 54
45 to 50
Processing batch 11 of 54
50 to 55
Processing batch 12 of 54
55 to 60
Processing batch 13 of 54
60 to 65
Processing batch 14 of 54
65 to 70
Processing batch 15 of 54
70 to 75
Processing batch 16 of 54
75 to 80
Processing batch 17 of 54
80 to 85
Processing batch 18 of 54
85 to 90
Processing batch 19 of 54
90 to 95
Processing batch 20 of 54
95 to 100
Processing batch 21 of 54
100 to 105
Processing batch 22 of 54
105 to 110
Processing batch 23 of 54
110 to 115
Processing batch 24 of 54
115 to 120
Processing batch 25 of 54
120 to 125
Processing batch 26 of 54
125 to 130
Processing batch 27 of 54
130 to 135
Processing batch 28 of 54
135 to 140
Processing batc

In [15]:
print(general_service_descriptions.general_service_description_output[251].general_service_description)

Comprehensive passport services including processing of new passport applications, renewals, and related documentation services. Services are delivered through a network of dedicated passport processing centers and auxiliary facilities.


In [14]:
for i, general_service_description in enumerate(general_service_descriptions.general_service_description_output):
    print(i)
    print(general_service_description.company_service_description)
    print(general_service_description.general_service_description)
    print("\n")

0
A&M is a global professional services firm that provides advisory, business performance improvement and turnaround management services across six continents. They help organizations tackle complex business issues, boost operating performance and maximize stakeholder value. The firm specializes in providing practical solutions to unique problems of companies, investors and government entities.
Professional advisory services focusing on business performance improvement and turnaround management. Services include addressing complex business challenges, enhancing operational efficiency, and maximizing value. Solutions are provided for organizational problems across various sectors including corporate, investment, and governmental domains.


1
Comprehensive tax preparation and financial services including business services (payroll, bookkeeping, sales & use tax), individual tax services, incorporation services (Corporation, LLC, Non-Profit), financial planning (retirement, college & educa

In [20]:
for i, general_service_description in enumerate(general_service_descriptions.general_service_description_output):
    await update_service_general_descriptions(general_service_description)